# AAK for spanish data

### DATA

We will use data from INE database. Specifically, the Make and Use (Origen y Destino) tables for industries are used in order to create the $Input$ and $Output$ matrices. Additionally, we take the nominal added value of each industry and transform it to with the GPD deflator. Later, some more data will be extracted, which will be used to analyze some shocks, to be determined.

#### Input Data and Libraries

In [52]:
import numpy as np
import pandas as pd
import math as mt
from sklearn import cross_decomposition
from linearmodels import PanelOLS

In [81]:
tio=pd.read_csv("tablainput_output.csv",delimiter=";")
tio=tio.set_index("Unnamed: 0")
nominal_vadd_ind=pd.read_csv('nominal_vadd_ind.csv',delimiter=";")
nominal_vadd_ind=nominal_vadd_ind.set_index("indice")

In [82]:
nominal_vadd_ind.head()

,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,...,2004,2003,2002,2001,2000,1999,1998,1997,1996,1995
indice,,,,,,,,,,,,,,,,,,,,,
1,29925,28928,27138,23797,24480,22620,22931,23976,22114,23826,...,23885,24568,23128,22853,21316,19935,20422,19918,19163,15778
2,995,966,968,956,1049,958,1085,981,981,1483,...,1580,1573,1550,1418,1448,1211,1137,962,847,774
3,1633,1580,1370,1426,1228,1254,1230,1122,1130,970,...,1421,1386,1474,1509,1500,1595,1517,1460,1461,1340
16,1912,1659,1602,1473,1493,1604,1805,2014,2196,2987,...,2648,2619,2601,2558,2531,2425,2307,2124,2030,1925
17,3773,3474,3392,3046,3447,3518,3689,3586,3252,3555,...,3205,3160,3218,3167,2981,2809,2781,2726,2635,2798


La TIO y las series de valor añadido (nominal) tienen diferentes índices pero entre los que se puede aplicar una biyección _traductora_. Cargamos el diccionario con las conversiones y las aplicamos mediante una función que definimos ahora para cambiar el índice de 'nominal_vadd_ind' de acuerdo a 'tio' y no al revés.

In [83]:
indice=pd.read_csv('indice.csv',delimiter=";")
indice=indice.set_index('vad')


In [84]:
indicenom=nominal_vadd_ind.index
ac=[]
for i in indicenom:
    ac.append(indice['tio'][i])
# lo añadimos
nominal_vadd_ind['indicetio']=ac
nominal_vadd_ind=nominal_vadd_ind.set_index('indicetio').sort_index()

In [85]:
nominal_vadd_ind.head() #probablemente haya que hacer una traspocición y 
                        #cambiar el orden de los años

,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,...,2004,2003,2002,2001,2000,1999,1998,1997,1996,1995
indicetio,,,,,,,,,,,,,,,,,,,,,
1,29925,28928,27138,23797,24480,22620,22931,23976,22114,23826,...,23885,24568,23128,22853,21316,19935,20422,19918,19163,15778
2,995,966,968,956,1049,958,1085,981,981,1483,...,1580,1573,1550,1418,1448,1211,1137,962,847,774
3,1633,1580,1370,1426,1228,1254,1230,1122,1130,970,...,1421,1386,1474,1509,1500,1595,1517,1460,1461,1340
4,2049,1474,1483,1625,1833,2292,2587,2639,2585,3264,...,2874,2678,2445,2222,1765,1708,1710,1827,2172,2389
5,26255,25045,24496,24309,24376,24829,25399,24546,24168,24415,...,18398,17437,16098,15281,14504,14452,13725,13366,12869,12565


### Creation of Input and Output tables

As we know, the coefficients we need are: $a_{ij}=\dfrac{Sales_{j \rightarrow i}}{Sales_i}$

In [86]:
tio.head()

,1,2,3,4,5,6,7,8,9,10,...,56,57,58,59,60,61,62,63,64,total ventas
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
1,871.0,0.1,0.0,0.2,"27,665.3",674.2,3.7,23.8,0.0,0.0,...,60.1,20.2,2.3,25.4,0.8,0.0,0.0,0.0,0.0,"31,495.9"
2,10.4,796.7,0.0,1.2,25.7,0.0,72.9,227.0,0.0,0.0,...,0.0,0.0,0.3,0.0,0.0,0.0,0.0,0.0,0.0,"1,275.5"
3,0.0,0.0,44.6,0.0,203.3,0.0,0.0,0.0,0.0,0.0,...,5.9,9.5,0.3,0.0,0.0,0.0,0.0,0.0,0.0,297.7
4,0.0,0.0,3.0,53.1,94.0,2.6,0.3,71.3,1.7,"16,960.4",...,0.0,0.0,1.3,3.2,0.0,0.1,0.8,0.0,0.0,"28,121.2"
5,"7,371.8",0.7,164.3,20.5,"23,595.3",194.5,0.1,58.8,0.0,8.8,...,394.3,382.3,212.6,225.5,11.8,0.0,66.0,0.0,0.0,"53,265.7"


In [87]:
A_in=pd.DataFrame()
for i in tio.index[:-1]:
      A_in[i]=np.zeros(len(tio.index[:-1]))
A_in["ind"]=tio.index[:-1]
A_in=A_in.set_index("ind")

In [88]:
for j in tio.columns:
    result=[]
    for i in tio.index:
        try:
            result.append(tio[j][i].replace(",",""))
        except:
            result.append(tio[j][i])
    tio[j]=result
for i in A_in.index[:-1]:
    for j in A_in.index[:-1]:
        try:
            sales=float(tio[i][j])/float(tio["total ventas"][i])
        except:
            sales=0
        A_in[j][i]=sales

In [89]:
A_in.head()

,1,2,3,4,5,6,7,8,9,10,...,55,56,57,58,59,60,61,62,63,64
ind,,,,,,,,,,,,,,,,,,,,,
1,0.027654,0.000330,0.000000,0.000000,0.234056,0.000248,0.000752,0.000111,0.000000,0.014808,...,0.000206,0.000000,0.0,0.000032,0.000000,0.000254,0.000549,0.000000,0.0,0.0
2,0.000078,0.624618,0.000000,0.000000,0.000549,0.000000,0.000000,0.000000,0.000000,0.009173,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
3,0.000000,0.000000,0.149815,0.010077,0.551898,0.136379,0.043332,0.001008,0.000000,0.585489,...,0.003023,0.007054,0.0,0.005039,0.002351,0.101780,0.008734,0.003023,0.0,0.0
4,0.000007,0.000043,0.000000,0.001888,0.000729,0.000050,0.001138,0.000124,0.000011,0.002375,...,0.000156,0.000178,0.0,0.000004,0.000004,0.000021,0.000139,0.000068,0.0,0.0
5,0.519383,0.000482,0.003817,0.001765,0.442974,0.000462,0.005163,0.033802,0.007817,0.001243,...,0.002409,0.001795,0.0,0.000377,0.001187,0.001476,0.000417,0.000723,0.0,0.0


In [90]:
A_inor=A_in.copy()
A_inor.to_csv("A_in_com.csv")

In [91]:
A_out=A_in.copy()
for i in A_in.index[:-1]:
    for j in A_in.index[:-1]:
        try:
                sales=float(tio[j][i])/float(tio["total ventas"][i])
        except:
                sales=0
        A_out[j][i]=sales

In [92]:
A_out.head()

,1,2,3,4,5,6,7,8,9,10,...,55,56,57,58,59,60,61,62,63,64
ind,,,,,,,,,,,,,,,,,,,,,
1,0.027654,0.000003,0.000000,0.000006,0.878378,0.021406,0.000117,0.000756,0.00000,0.000000,...,0.003572,0.001908,0.000641,0.000073,0.000806,0.000025,0.000000,0.000000,0.0,0.0
2,0.008154,0.624618,0.000000,0.000941,0.020149,0.000000,0.057154,0.177969,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000235,0.000000,0.000000,0.000000,0.000000,0.0,0.0
3,0.000000,0.000000,0.149815,0.000000,0.682902,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.071549,0.019819,0.031911,0.001008,0.000000,0.000000,0.000000,0.000000,0.0,0.0
4,0.000000,0.000000,0.000107,0.001888,0.003343,0.000092,0.000011,0.002535,0.00006,0.603118,...,0.000018,0.000000,0.000000,0.000046,0.000114,0.000000,0.000004,0.000028,0.0,0.0
5,0.138397,0.000013,0.003085,0.000385,0.442974,0.003652,0.000002,0.001104,0.00000,0.000165,...,0.006143,0.007403,0.007177,0.003991,0.004233,0.000222,0.000000,0.001239,0.0,0.0


In [212]:
A_in.head()

,1,2,3,4,5,6,7,8,9,10,...,55,56,57,58,59,60,61,62,63,64
ind,,,,,,,,,,,,,,,,,,,,,
1,0.027654,0.000330,0.000000,0.000000,0.234056,0.000248,0.000752,0.000111,0.000000,0.014808,...,0.000206,0.000000,0.0,0.000032,0.000000,0.000254,0.000549,0.000000,0.0,0.0
2,0.000078,0.624618,0.000000,0.000000,0.000549,0.000000,0.000000,0.000000,0.000000,0.009173,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
3,0.000000,0.000000,0.149815,0.010077,0.551898,0.136379,0.043332,0.001008,0.000000,0.585489,...,0.003023,0.007054,0.0,0.005039,0.002351,0.101780,0.008734,0.003023,0.0,0.0
4,0.000007,0.000043,0.000000,0.001888,0.000729,0.000050,0.001138,0.000124,0.000011,0.002375,...,0.000156,0.000178,0.0,0.000004,0.000004,0.000021,0.000139,0.000068,0.0,0.0
5,0.519383,0.000482,0.003817,0.001765,0.442974,0.000462,0.005163,0.033802,0.007817,0.001243,...,0.002409,0.001795,0.0,0.000377,0.001187,0.001476,0.000417,0.000723,0.0,0.0


In [213]:
A_out.head()

,1,2,3,4,5,6,7,8,9,10,...,55,56,57,58,59,60,61,62,63,64
ind,,,,,,,,,,,,,,,,,,,,,
1,0.027654,0.000003,0.000000,0.000006,0.878378,0.021406,0.000117,0.000756,0.00000,0.000000,...,0.003572,0.001908,0.000641,0.000073,0.000806,0.000025,0.000000,0.000000,0.0,0.0
2,0.008154,0.624618,0.000000,0.000941,0.020149,0.000000,0.057154,0.177969,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000235,0.000000,0.000000,0.000000,0.000000,0.0,0.0
3,0.000000,0.000000,0.149815,0.000000,0.682902,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.071549,0.019819,0.031911,0.001008,0.000000,0.000000,0.000000,0.000000,0.0,0.0
4,0.000000,0.000000,0.000107,0.001888,0.003343,0.000092,0.000011,0.002535,0.00006,0.603118,...,0.000018,0.000000,0.000000,0.000046,0.000114,0.000000,0.000004,0.000028,0.0,0.0
5,0.138397,0.000013,0.003085,0.000385,0.442974,0.003652,0.000002,0.001104,0.00000,0.000165,...,0.006143,0.007403,0.007177,0.003991,0.004233,0.000222,0.000000,0.001239,0.0,0.0


In [386]:
A_incom=A_in.copy()
A_outcom=A_out.copy()
"""
for i in A_incom.index:
    A_incom[i][i]=0
    A_outcom[i][i]=0
    """

'\nfor i in A_incom.index:\n    A_incom[i][i]=0\n    A_outcom[i][i]=0\n    '

In [387]:
A_incom=A_incom.drop("63",axis=1)
A_outcom=A_outcom.drop("63",axis=1)
A_incom=A_incom.drop("64",axis=1)
A_outcom=A_outcom.drop("64",axis=1)
A_incom=A_incom.drop("64",axis=0)
A_outcom=A_outcom.drop("64",axis=0)
A_incom=A_incom.drop("63",axis=0)
A_outcom=A_outcom.drop("63",axis=0)

In [388]:

cca = cross_decomposition.CCA(n_components=1)
U_c, V_c = cca.fit_transform(A_incom, A_outcom)

result = np.corrcoef(U_c.T, V_c.T)[0,1]

In [389]:
result #logico

1.0

In [390]:
vA=[]
for i in A_incom.index:
    for j in A_incom.columns:
        vA.append(A_incom[i][j])
vB=[]
for i in A_outcom.index:
    for j in A_outcom.columns:
        vB.append(A_outcom[i][j])

In [391]:
A_incom.head()

,1,2,3,4,5,6,7,8,9,10,...,53,54,55,56,57,58,59,60,61,62
ind,,,,,,,,,,,,,,,,,,,,,
1,0.027654,0.000330,0.000000,0.000000,0.234056,0.000248,0.000752,0.000111,0.000000,0.014808,...,0.002832,0.000159,0.000206,0.000000,0.0,0.000032,0.000000,0.000254,0.000549,0.000000
2,0.000078,0.624618,0.000000,0.000000,0.000549,0.000000,0.000000,0.000000,0.000000,0.009173,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.149815,0.010077,0.551898,0.136379,0.043332,0.001008,0.000000,0.585489,...,0.018475,0.093719,0.003023,0.007054,0.0,0.005039,0.002351,0.101780,0.008734,0.003023
4,0.000007,0.000043,0.000000,0.001888,0.000729,0.000050,0.001138,0.000124,0.000011,0.002375,...,0.003517,0.000128,0.000156,0.000178,0.0,0.000004,0.000004,0.000021,0.000139,0.000068
5,0.519383,0.000482,0.003817,0.001765,0.442974,0.000462,0.005163,0.033802,0.007817,0.001243,...,0.020940,0.000389,0.002409,0.001795,0.0,0.000377,0.001187,0.001476,0.000417,0.000723


In [392]:
len(vB)

3844

In [393]:
a2=pd.DataFrame()
a2[0]=vA
a2[1]=vB

In [397]:
a2.corr(method="pearson")

,0,1
0,1.00000,0.26473
1,0.26473,1.00000


In [398]:
a2.corr(method="spearman")

,0,1
0,1.000000,0.225511
1,0.225511,1.000000


### Federal Spending Shocks

The easiest shock for the available data for the Spanish Economy and that exists in the Acemoglu paper is the Federal Spending one. It is quite an inmportan shock, though. The Shock equations are: (Gov=Fed)
 
 $GovSales\%=\dfrac{Sales_{i\rightarrow Gov}}{Sales_i}$ 
  
  $GovShock_{i,t}=GovSales\%_i^{T^*}·\Delta \ln GovSpending_{t-1}$
   
 
  
 $Downstream_{i,t}^{Gov}=\sum_j(Input\%_{j \rightarrow i}^{T^*}-1_{j=i})GovShock_{j,t}$
  
where $T^*$ is a basis year. 

In [362]:
tiog=pd.read_csv('govtio.csv',delimiter=";")
tiog=tiog.set_index("Unnamed: 0")

In [363]:
tiog.head() # para cada industria, el total de ventas y cuántas van a AAPP

,Total demanda final,Gasto en consumo final de las Administraciones Públicas,Ratio
Unnamed: 0,,,
1,"24,286.8",10.7,0.000441
2,817.5,64.0,0.078287
3,"3,913.7",0.0,0.000000
4,"2,073.5",97.0,0.046781
5,"82,007.9",0.0,0.000000


Tenemos datos de 1995 a 2017 así que serán estos los que tomaremos para $GovSpending_t$ (mejor si tenemos 1994, 2018)

In [364]:
gasto=pd.read_csv('gastogov.csv',delimiter=";")
gasto.columns=["year","spending"]
gasto=gasto.set_index("year")
#tranformamos a l_gasto:
l_g=[]
for i in gasto["spending"]:
    l_g.append(mt.log(float(i)))
#transformaos a d_l_gasto:
d_l_g=[np.nan]
anterior=l_g[0]
for i in l_g[1:]:
    d_l_g.append(i-anterior)
    anterior=i
gasto["d_l_g"]=d_l_g

In [365]:
gasto

,spending,d_l_g
year,,
1995,81127,NaN
1996,85548,0.053062
1997,88310,0.031776
1998,93728,0.059544
1999,100025,0.065023
2000,108177,0.078349
2001,115977,0.069623
2002,124608,0.071781
2003,134593,0.077083


In [366]:
# then, for each industry and year we have a value of this shock, in 
# a matrix called matshock:

In [367]:
gasto=gasto.drop(1995)

In [368]:
gasto

,spending,d_l_g
year,,
1996,85548,0.053062
1997,88310,0.031776
1998,93728,0.059544
1999,100025,0.065023
2000,108177,0.078349
2001,115977,0.069623
2002,124608,0.071781
2003,134593,0.077083
2004,147556,0.091952


In [369]:
matshock=pd.DataFrame()

In [370]:
for ind in tiog.index:
    ac=[]
    for year in gasto.index:
        ac.append(gasto["d_l_g"][year]*tiog["Ratio"][ind])
    matshock[ind]=ac
matshock["year"]=range(1996,2019)
matshock=matshock.set_index("year")
        

In [371]:
matshock

,1,2,3,4,5,6,7,8,9,10,...,55,56,57,58,59,60,61,62,63,64
year,,,,,,,,,,,,,,,,,,,,,
1996,2.337738e-05,0.004154,0.0,0.002482,0.0,7.070537e-07,0.0,0.000115,0.0,0.0,...,0.052142,0.039758,0.039750,0.029964,0.016609,0.013956,0.000081,0.0,0.000178,0.0
1997,1.399940e-05,0.002488,0.0,0.001486,0.0,4.234148e-07,0.0,0.000069,0.0,0.0,...,0.031225,0.023809,0.023804,0.017944,0.009946,0.008357,0.000049,0.0,0.000107,0.0
1998,2.623307e-05,0.004662,0.0,0.002785,0.0,7.934247e-07,0.0,0.000130,0.0,0.0,...,0.058511,0.044614,0.044606,0.033625,0.018638,0.015660,0.000091,0.0,0.000200,0.0
1999,2.864720e-05,0.005091,0.0,0.003042,0.0,8.664404e-07,0.0,0.000142,0.0,0.0,...,0.063896,0.048720,0.048711,0.036719,0.020354,0.017101,0.000100,0.0,0.000218,0.0
2000,3.451797e-05,0.006134,0.0,0.003665,0.0,1.044003e-06,0.0,0.000171,0.0,0.0,...,0.076990,0.058705,0.058694,0.044244,0.024525,0.020606,0.000120,0.0,0.000263,0.0
2001,3.067379e-05,0.005451,0.0,0.003257,0.0,9.277351e-07,0.0,0.000152,0.0,0.0,...,0.068416,0.052167,0.052157,0.039317,0.021793,0.018311,0.000107,0.0,0.000234,0.0
2002,3.162445e-05,0.005620,0.0,0.003358,0.0,9.564879e-07,0.0,0.000156,0.0,0.0,...,0.070536,0.053784,0.053773,0.040535,0.022469,0.018879,0.000110,0.0,0.000241,0.0
2003,3.396020e-05,0.006035,0.0,0.003606,0.0,1.027133e-06,0.0,0.000168,0.0,0.0,...,0.075746,0.057756,0.057745,0.043529,0.024128,0.020273,0.000118,0.0,0.000259,0.0
2004,4.051136e-05,0.007199,0.0,0.004302,0.0,1.225274e-06,0.0,0.000200,0.0,0.0,...,0.090358,0.068897,0.068885,0.051926,0.028783,0.024184,0.000141,0.0,0.000309,0.0


In [372]:
#now downstrean and upstream

 $Downstream_{i,t}^{Gov}=\sum_j(Input\%_{j \rightarrow i}^{T^*}-1_{j=i})GovShock_{j,t}$

In [373]:
#now downstrean and upstream
# necesitamos H=(Id-A)^-1

Id_A=A_in.copy()
for i in Id_A.index:
    for j in Id_A.columns:
        if i!=j:
            Id_A[j][i]=0
        if i==j:
            Id_A[j][i]=1
Id_A2=Id_A-A_in
h_in = pd.DataFrame(np.linalg.pinv(Id_A2.values), Id_A2.columns, Id_A2.index)
Amid=h_in-Id_A
downstream=matshock.copy()

A_n=A_in
Amid=A_n
for i in range(200):
    A_n=A_n.dot(A_n)
    Amid=Amid+A_n
Amid=Amid-Id_A

In [374]:
# downstream shock

In [375]:
def valorprop(industria,anyo):
    valinput=Amid.iloc[industria-1].values
    valshock=matshock.loc[anyo]
    valor=sum(valinput*valshock)
    return valor

In [376]:
downstream=matshock.copy()
for year in downstream.index:
    for ind in downstream.columns:
        downstream[ind][year]=valorprop(int(ind),year)

In [377]:
downstream

,1,2,3,4,5,6,7,8,9,10,...,55,56,57,58,59,60,61,62,63,64
year,,,,,,,,,,,,,,,,,,,,,
1996,0.011092,0.007308,0.099607,-0.001379,0.024923,0.021255,0.008073,0.010740,0.007695,0.012859,...,-0.011331,0.083094,0.810654,0.043420,0.010322,0.010482,0.011531,0.027350,-0.000178,0.0
1997,0.006643,0.004376,0.059649,-0.000826,0.014925,0.012728,0.004834,0.006432,0.004608,0.007701,...,-0.006786,0.049761,0.485455,0.026002,0.006181,0.006277,0.006905,0.016378,-0.000107,0.0
1998,0.012447,0.008201,0.111775,-0.001547,0.027968,0.023851,0.009059,0.012052,0.008635,0.014430,...,-0.012715,0.093245,0.909680,0.048725,0.011583,0.011763,0.012940,0.030691,-0.000200,0.0
1999,0.013593,0.008955,0.122061,-0.001690,0.030542,0.026046,0.009893,0.013161,0.009429,0.015758,...,-0.013885,0.101826,0.993394,0.053209,0.012649,0.012845,0.014130,0.033515,-0.000218,0.0
2000,0.016378,0.010790,0.147075,-0.002036,0.036801,0.031383,0.011920,0.015858,0.011362,0.018988,...,-0.016731,0.122694,1.196974,0.064113,0.015241,0.015478,0.017026,0.040383,-0.000263,0.0
2001,0.014554,0.009589,0.130696,-0.001809,0.032702,0.027888,0.010593,0.014092,0.010097,0.016873,...,-0.014868,0.109029,1.063670,0.056973,0.013543,0.013754,0.015130,0.035886,-0.000234,0.0
2002,0.015005,0.009886,0.134746,-0.001865,0.033716,0.028753,0.010921,0.014529,0.010409,0.017396,...,-0.015328,0.112409,1.096636,0.058738,0.013963,0.014180,0.015599,0.036998,-0.000241,0.0
2003,0.016114,0.010616,0.144699,-0.002003,0.036206,0.030876,0.011728,0.015602,0.011178,0.018681,...,-0.016461,0.120711,1.177632,0.063077,0.014994,0.015228,0.016751,0.039731,-0.000259,0.0
2004,0.019222,0.012664,0.172612,-0.002389,0.043190,0.036833,0.013990,0.018612,0.013335,0.022285,...,-0.019636,0.143997,1.404806,0.075245,0.017887,0.018165,0.019982,0.047395,-0.000309,0.0


In [378]:
#upstream

In [379]:
Id_A=A_out.copy()
for i in Id_A.index:
    for j in Id_A.columns:
        if i!=j:
            Id_A[j][i]=0
        if i==j:
            Id_A[j][i]=1
Id_A2=Id_A-A_out
h_out = pd.DataFrame(np.linalg.pinv(Id_A2.values), Id_A2.columns, Id_A2.index)
Amid=h_out-Id_A
A_n=A_out
Amid=A_n
for i in range(200):
    A_n=A_n.dot(A_n)
    Amid=Amid+A_n
Amid=Amid-Id_A
downstream2=matshock.copy()

In [380]:
def valorprop(industria,anyo):
    valinput=Amid.iloc[industria-1].values
    valshock=matshock.loc[anyo]
    valor=sum(valinput*valshock)
    return valor

In [381]:
downstream2=matshock.copy()
for year in downstream2.index:
    for ind in downstream2.columns:
        downstream2[ind][year]=valorprop(int(ind),year)

In [382]:
upstream=downstream2
upstream

,1,2,3,4,5,6,7,8,9,10,...,55,56,57,58,59,60,61,62,63,64
year,,,,,,,,,,,,,,,,,,,,,
1996,0.115244,0.112865,0.124235,0.112261,0.117437,0.118767,0.113233,0.117886,0.121807,0.119422,...,0.075673,0.114797,0.161298,0.098276,0.112296,0.119922,0.131140,0.136039,-0.000178,0.0
1997,0.069013,0.067589,0.074398,0.067227,0.070326,0.071123,0.067809,0.070596,0.072944,0.071515,...,0.045316,0.068745,0.096593,0.058852,0.067248,0.071815,0.078532,0.081466,-0.000107,0.0
1998,0.129322,0.126652,0.139412,0.125974,0.131782,0.133275,0.127065,0.132287,0.136687,0.134010,...,0.084917,0.128820,0.181002,0.110281,0.126014,0.134571,0.147159,0.152657,-0.000200,0.0
1999,0.141223,0.138308,0.152241,0.137567,0.143910,0.145540,0.138758,0.144461,0.149266,0.146342,...,0.092731,0.140675,0.197659,0.120430,0.137610,0.146956,0.160702,0.166705,-0.000218,0.0
2000,0.170165,0.166652,0.183440,0.165759,0.173402,0.175366,0.167194,0.174066,0.179855,0.176333,...,0.111735,0.169504,0.238166,0.145110,0.165811,0.177072,0.193635,0.200869,-0.000263,0.0
2001,0.151214,0.148092,0.163011,0.147299,0.154090,0.155836,0.148574,0.154680,0.159825,0.156695,...,0.099291,0.150627,0.211642,0.128950,0.147345,0.157352,0.172070,0.178499,-0.000234,0.0
2002,0.155900,0.152682,0.168063,0.151864,0.158866,0.160666,0.153179,0.159474,0.164779,0.161551,...,0.102369,0.155295,0.218201,0.132946,0.151912,0.162228,0.177403,0.184031,-0.000241,0.0
2003,0.167415,0.163959,0.180476,0.163080,0.170600,0.172532,0.164493,0.171253,0.176949,0.173483,...,0.109930,0.166765,0.234317,0.142765,0.163132,0.174210,0.190506,0.197623,-0.000259,0.0
2004,0.199711,0.195587,0.215291,0.194540,0.203510,0.205815,0.196225,0.204289,0.211084,0.206949,...,0.131136,0.198935,0.279519,0.170306,0.194601,0.207817,0.227256,0.235746,-0.000309,0.0


### Estimation (After stacking)

Primero probaremos en niveles y luego en diferencias, con ln en el caso de Yt

$ Y_{i,t}=\delta_t+\psi Y_{i,t-1}+\beta^{own}Shock_{i,t-1}+\beta^{upstream}Upstream_{i,t-1}+\beta^{downstream}Downstream_{i,t-1}+\varepsilon_{i,t}\;\;\forall_i$

In [383]:
yt=nominal_vadd_ind.transpose()
yt=yt.sort_index()
yt["indice"]=range(1995,2018)
yt=yt.set_index("indice")

In [384]:
yt.head()

indicetio,1,2,3,4,5,6,7,8,9,10,...,55,56,57,58,59,60,61,62,63,64
indice,,,,,,,,,,,,,,,,,,,,,
1995,15778,774,1340,2389,12565,5972,1925,2798,1849,1952,...,21996,17893,3787,3628,2902,1725,431,4622,4509,0
1996,19163,847,1461,2172,12869,6407,2030,2635,2132,1970,...,23756,18947,4060,3884,3122,1832,465,4709,4662,0
1997,19918,962,1460,1827,13366,6875,2124,2726,2399,2338,...,25037,19367,4208,4049,3276,1955,502,4792,4798,0
1998,20422,1137,1517,1710,13725,7308,2307,2781,2633,2349,...,26711,20391,4479,4369,3568,2071,550,4993,5159,0
1999,19935,1211,1595,1708,14452,7435,2425,2809,2838,1700,...,28514,21648,4839,4714,3816,2241,620,5161,5572,0


In [385]:
yt_1=yt.copy()
for i in yt_1.columns:
    ac=[np.nan]
    anterior=yt_1[i][1995]
    for j in yt_1[i][1:]:
        ac.append(anterior)
        anterior=j
    yt_1[i]=ac

Aquí tenemos $Y_{t-1}$, hacemos lo mismo para hallar $Upstream_{t-1}$ y $Downstream_{t-1}$ y $OwnShock_{t-1}$

In [386]:
yt_1.head()

indicetio,1,2,3,4,5,6,7,8,9,10,...,55,56,57,58,59,60,61,62,63,64
indice,,,,,,,,,,,,,,,,,,,,,
1995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,15778.0,774.0,1340.0,2389.0,12565.0,5972.0,1925.0,2798.0,1849.0,1952.0,...,21996.0,17893.0,3787.0,3628.0,2902.0,1725.0,431.0,4622.0,4509.0,0.0
1997,19163.0,847.0,1461.0,2172.0,12869.0,6407.0,2030.0,2635.0,2132.0,1970.0,...,23756.0,18947.0,4060.0,3884.0,3122.0,1832.0,465.0,4709.0,4662.0,0.0
1998,19918.0,962.0,1460.0,1827.0,13366.0,6875.0,2124.0,2726.0,2399.0,2338.0,...,25037.0,19367.0,4208.0,4049.0,3276.0,1955.0,502.0,4792.0,4798.0,0.0
1999,20422.0,1137.0,1517.0,1710.0,13725.0,7308.0,2307.0,2781.0,2633.0,2349.0,...,26711.0,20391.0,4479.0,4369.0,3568.0,2071.0,550.0,4993.0,5159.0,0.0


In [387]:
downstream_1=downstream.copy()
for i in downstream_1.columns:
    ac=[np.nan]
    anterior=downstream_1[i][1996]
    for j in downstream_1[i][1:]:
        ac.append(anterior)
        anterior=j
    downstream_1[i]=ac

In [388]:
upstream_1=upstream.copy()
for i in upstream_1.columns:
    ac=[np.nan]
    anterior=upstream_1[i][1996]
    for j in upstream_1[i][1:]:
        ac.append(anterior)
        anterior=j
    upstream_1[i]=ac

In [389]:
matshock_1=matshock.copy()
for i in matshock_1.columns:
    ac=[np.nan]
    anterior=matshock_1[i][1996]
    for j in matshock_1[i][1:]:
        ac.append(anterior)
        anterior=j
    matshock_1[i]=ac

In [390]:
downstream_1

,1,2,3,4,5,6,7,8,9,10,...,55,56,57,58,59,60,61,62,63,64
year,,,,,,,,,,,,,,,,,,,,,
1996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997,0.011092,0.007308,0.099607,-0.001379,0.024923,0.021255,0.008073,0.010740,0.007695,0.012859,...,-0.011331,0.083094,0.810654,0.043420,0.010322,0.010482,0.011531,0.027350,-0.000178,0.0
1998,0.006643,0.004376,0.059649,-0.000826,0.014925,0.012728,0.004834,0.006432,0.004608,0.007701,...,-0.006786,0.049761,0.485455,0.026002,0.006181,0.006277,0.006905,0.016378,-0.000107,0.0
1999,0.012447,0.008201,0.111775,-0.001547,0.027968,0.023851,0.009059,0.012052,0.008635,0.014430,...,-0.012715,0.093245,0.909680,0.048725,0.011583,0.011763,0.012940,0.030691,-0.000200,0.0
2000,0.013593,0.008955,0.122061,-0.001690,0.030542,0.026046,0.009893,0.013161,0.009429,0.015758,...,-0.013885,0.101826,0.993394,0.053209,0.012649,0.012845,0.014130,0.033515,-0.000218,0.0
2001,0.016378,0.010790,0.147075,-0.002036,0.036801,0.031383,0.011920,0.015858,0.011362,0.018988,...,-0.016731,0.122694,1.196974,0.064113,0.015241,0.015478,0.017026,0.040383,-0.000263,0.0
2002,0.014554,0.009589,0.130696,-0.001809,0.032702,0.027888,0.010593,0.014092,0.010097,0.016873,...,-0.014868,0.109029,1.063670,0.056973,0.013543,0.013754,0.015130,0.035886,-0.000234,0.0
2003,0.015005,0.009886,0.134746,-0.001865,0.033716,0.028753,0.010921,0.014529,0.010409,0.017396,...,-0.015328,0.112409,1.096636,0.058738,0.013963,0.014180,0.015599,0.036998,-0.000241,0.0
2004,0.016114,0.010616,0.144699,-0.002003,0.036206,0.030876,0.011728,0.015602,0.011178,0.018681,...,-0.016461,0.120711,1.177632,0.063077,0.014994,0.015228,0.016751,0.039731,-0.000259,0.0


In [391]:
matshock_1=matshock.copy()
for i in matshock_1.columns:
    ac=[np.nan]
    anterior=matshock_1[i][1996]
    for j in matshock_1[i][1:]:
        ac.append(anterior)
        anterior=j
    matshock_1[i]=ac

In [392]:
matshock_1=matshock_1.drop(1996)
matshock_1=matshock_1.drop(2018)
downstream_1=downstream_1.drop(1996)
upstream_1  =  upstream_1.drop(1996)
yt_1=yt_1.drop(1995)
yt_1=yt_1.drop(1996)

In [400]:
downstream_1=downstream_1.drop(2018)
upstream_1=upstream_1.drop(2018)

In [401]:
yt=yt.drop(1995)
yt=yt.drop(1996)

#### now we stack

In [402]:
yt

indicetio,1,2,3,4,5,6,7,8,9,10,...,55,56,57,58,59,60,61,62,63,64
indice,,,,,,,,,,,,,,,,,,,,,
1997,19918,962,1460,1827,13366,6875,2124,2726,2399,2338,...,25037,19367,4208,4049,3276,1955,502,4792,4798,0
1998,20422,1137,1517,1710,13725,7308,2307,2781,2633,2349,...,26711,20391,4479,4369,3568,2071,550,4993,5159,0
1999,19935,1211,1595,1708,14452,7435,2425,2809,2838,1700,...,28514,21648,4839,4714,3816,2241,620,5161,5572,0
2000,21316,1448,1500,1765,14504,7572,2531,2981,3405,2713,...,30752,23471,4655,5114,4194,2407,685,5336,5809,0
2001,22853,1418,1509,2222,15281,8109,2558,3167,3446,2894,...,32985,24942,5180,5712,4545,2670,767,5428,6178,0
2002,23128,1550,1474,2445,16098,8153,2601,3218,3645,2783,...,34844,27245,6185,6262,4913,2990,844,5574,6529,0
2003,24568,1573,1386,2678,17437,8556,2619,3160,3678,3318,...,36866,30385,7290,6787,5249,3256,867,5781,7029,0
2004,23885,1580,1421,2874,18398,8337,2648,3205,3744,3483,...,39345,32915,8298,7381,5740,3636,904,5961,7492,0
2005,22751,1661,1267,2855,20505,8289,2783,3241,4149,3818,...,41422,35834,9676,8370,6400,4069,955,6290,8068,0


In [403]:
data=pd.DataFrame()

In [404]:
data["yt"]=[]
data["industry"]=[]
data["year"]=[]

In [405]:
for i in yt.columns:
    for j in yt.index:
        parcial=pd.DataFrame()
        parcial["yt"]=[yt[i][j]]
        parcial["industry"]=[i]
        parcial["year"]=[j]
        data=pd.concat([data,parcial])
data["indice"]=range(0,len(data))
data=data.set_index("indice")
ac=[]

In [406]:
data.head()

,yt,industry,year
indice,,,
0,19918.0,1.0,1997.0
1,20422.0,1.0,1998.0
2,19935.0,1.0,1999.0
3,21316.0,1.0,2000.0
4,22853.0,1.0,2001.0


In [407]:
ac=[]
for i in data["yt"]:
    try:
        ac.append(mt.log(i))
    except:
        ac.append(0)
data["yt"]=ac

for i in data.columns:
    ac=[]
    for j in data[i]:
        ac.append(float(j))
    data[i]=ac

data2=pd.DataFrame()
data2["upstream"]=[]
data2["industry"]=[]
data2["year"]=[]
for i in upstream_1.columns:
    for j in upstream_1.index:
        parcial=pd.DataFrame()
        parcial["upstream"]=[upstream_1[i][j]]
        parcial["industry"]=[i]
        parcial["year"]=[j]
        data2=pd.concat([data2,parcial])
data2["indice"]=range(0,len(data2))
data2=data2.set_index("indice")

data2.head()

,upstream,industry,year
indice,,,
0,0.115244,1.0,1997.0
1,0.069013,1.0,1998.0
2,0.129322,1.0,1999.0
3,0.141223,1.0,2000.0
4,0.170165,1.0,2001.0


In [408]:
for i in data.columns:
    ac=[]
    for j in data[i]:
        ac.append(float(j))
    data[i]=ac

In [409]:
data2=pd.DataFrame()
data2["upstream"]=[]
data2["industry"]=[]
data2["year"]=[]
for i in upstream_1.columns:
    for j in upstream_1.index:
        parcial=pd.DataFrame()
        parcial["upstream"]=[upstream_1[i][j]]
        parcial["industry"]=[i]
        parcial["year"]=[j]
        data2=pd.concat([data2,parcial])
data2["indice"]=range(0,len(data2))
data2=data2.set_index("indice")

In [410]:
data2.head()

,upstream,industry,year
indice,,,
0,0.115244,1.0,1997.0
1,0.069013,1.0,1998.0
2,0.129322,1.0,1999.0
3,0.141223,1.0,2000.0
4,0.170165,1.0,2001.0


In [411]:
for i in data2.columns:
    ac=[]
    for j in data2[i]:
        ac.append(float(j))
    data2[i]=ac

In [412]:
data3=pd.DataFrame()
data3["downstream"]=[]
data3["industry"]=[]
data3["year"]=[]
for i in downstream_1.columns:
    for j in downstream_1.index:
        parcial=pd.DataFrame()
        parcial["downstream"]=[downstream_1[i][j]]
        parcial["industry"]=[i]
        parcial["year"]=[j]
        data3=pd.concat([data3,parcial])
data3["indice"]=range(0,len(data3))
data3=data3.set_index("indice")

In [413]:
for i in data3.columns:
    ac=[]
    for j in data3[i]:
        ac.append(float(j))
    data3[i]=ac

In [414]:
data4=pd.DataFrame()
data4["ownshock"]=[]
data4["industry"]=[]
data4["year"]=[]
for i in matshock_1.columns:
    for j in matshock_1.index:
        parcial=pd.DataFrame()
        parcial["ownshock"]=[matshock_1[i][j]]
        parcial["industry"]=[i]
        parcial["year"]=[j]
        data4=pd.concat([data4,parcial])
data4["indice"]=range(0,len(data4))
data4=data4.set_index("indice")

In [415]:
for i in data4.columns:
    ac=[]
    for j in data4[i]:
        ac.append(float(j))
    data4[i]=ac

In [416]:
data4.head()

,ownshock,industry,year
indice,,,
0,0.000023,1.0,1997.0
1,0.000014,1.0,1998.0
2,0.000026,1.0,1999.0
3,0.000029,1.0,2000.0
4,0.000035,1.0,2001.0


In [417]:
data5=pd.DataFrame()
data5["yt_1"]=[]
data5["industry"]=[]
data5["year"]=[]
for i in yt_1.columns:
    for j in yt_1.index:
        parcial=pd.DataFrame()
        parcial["yt_1"]=[yt_1[i][j]]
        parcial["industry"]=[i]
        parcial["year"]=[j]
        data5=pd.concat([data5,parcial])
data5["indice"]=range(0,len(data5))
data5=data5.set_index("indice")

In [418]:
ac=[]
for i in data5["yt_1"]:
    try:
        ac.append(mt.log(i))
    except:
        ac.append(0)
data5["yt_1"]=ac

In [419]:
data2=data2.drop(["year","industry"],axis=1)
data3=data3.drop(["year","industry"],axis=1)
data4=data4.drop(["year","industry"],axis=1)
data5=data5.drop(["year","industry"],axis=1)

In [420]:
data=pd.concat([data,data2,data3,data4,data5],axis=1)

In [421]:
data.head()

,yt,industry,year,upstream,downstream,ownshock,yt_1
indice,,,,,,,
0,9.899379,1.0,1997.0,0.115244,0.011092,0.000023,9.860737
1,9.924368,1.0,1998.0,0.069013,0.006643,0.000014,9.899379
2,9.900232,1.0,1999.0,0.129322,0.012447,0.000026,9.924368
3,9.967213,1.0,2000.0,0.141223,0.013593,0.000029,9.900232
4,10.036838,1.0,2001.0,0.170165,0.016378,0.000035,9.967213


In [422]:
def dif_log(num1,num2):
    res=mt.log(num1)-mt.log(num2)
    return res

In [423]:
data.head()

,yt,industry,year,upstream,downstream,ownshock,yt_1
indice,,,,,,,
0,9.899379,1.0,1997.0,0.115244,0.011092,0.000023,9.860737
1,9.924368,1.0,1998.0,0.069013,0.006643,0.000014,9.899379
2,9.900232,1.0,1999.0,0.129322,0.012447,0.000026,9.924368
3,9.967213,1.0,2000.0,0.141223,0.013593,0.000029,9.900232
4,10.036838,1.0,2001.0,0.170165,0.016378,0.000035,9.967213


In [424]:
data2=data.copy()
for i in ["yt","yt_1"]:
    ac=[0]
    anterior=data2[i][0]
    for j in data2[i][1:]:
        ac.append(j-anterior)
        anterior=j
    data[i]=ac

In [425]:
data

,yt,industry,year,upstream,downstream,ownshock,yt_1
indice,,,,,,,
0,0.000000,1.0,1997.0,0.115244,0.011092,2.337738e-05,0.000000
1,0.024989,1.0,1998.0,0.069013,0.006643,1.399940e-05,0.038643
2,-0.024136,1.0,1999.0,0.129322,0.012447,2.623307e-05,0.024989
3,0.066981,1.0,2000.0,0.141223,0.013593,2.864720e-05,-0.024136
4,0.069624,1.0,2001.0,0.170165,0.016378,3.451797e-05,0.066981
5,0.011962,1.0,2002.0,0.151214,0.014554,3.067379e-05,0.069624
6,0.060401,1.0,2003.0,0.155900,0.015005,3.162445e-05,0.011962
7,-0.028194,1.0,2004.0,0.167415,0.016114,3.396020e-05,0.060401
8,-0.048642,1.0,2005.0,0.199711,0.019222,4.051136e-05,-0.028194


In [426]:
data=data[data["industry"]!=63]
data=data[data["industry"]!=64]

In [427]:
data=data[data["year"]!=1997]
data=data.set_index(["industry","year"])

In [429]:
data

yt  upstream  downstream      ownshock      yt_1
industry year                                                          
1.0      1998.0  0.024989  0.069013    0.006643  1.399940e-05  0.038643
         1999.0 -0.024136  0.129322    0.012447  2.623307e-05  0.024989
         2000.0  0.066981  0.141223    0.013593  2.864720e-05 -0.024136
         2001.0  0.069624  0.170165    0.016378  3.451797e-05  0.066981
         2002.0  0.011962  0.151214    0.014554  3.067379e-05  0.069624
         2003.0  0.060401  0.155900    0.015005  3.162445e-05  0.011962
         2004.0 -0.028194  0.167415    0.016114  3.396020e-05  0.060401
         2005.0 -0.048642  0.199711    0.019222  4.051136e-05 -0.028194
         2006.0 -0.065902  0.185682    0.017872  3.766569e-05 -0.048642
         2007.0  0.117219  0.175679    0.016909  3.563655e-05 -0.065902
         2008.0 -0.005149  0.192650    0.018542  3.907905e-05  0.117219
         2009.0 -0.074567  0.200523    0.019300  4.067614e-05 -0.005149
         2010.0  0.080842  0.119912    0.011541  2.432411e-05 -0.074567
         2011.0 -0.044564  0.006152    0.000592  1.247846e-06  0.080842
         2012.0 -0.013655 -0.014108   -0.001358 -2.861723e-06 -0.044564
         2013.0  0.079022 -0.141987   -0.013666 -2.880221e-05 -0.013655
         2014.0 -0.028297 -0.033256   -0.003201 -6.746053e-06  0.079022
         2015.0  0.131375 -0.001864   -0.000179 -3.780683e-07 -0.028297
         2016.0  0.063875  0.076147    0.007329  1.544651e-05  0.131375
         2017.0  0.033884  0.024364    0.002345  4.942245e-06  0.063875
2.0      1998.0  0.167134  0.067589    0.004376  2.487641e-03  0.127314
         1999.0  0.063053  0.126652    0.008201  4.661519e-03  0.167134
         2000.0  0.178737  0.138308    0.008955  5.090500e-03  0.063053
         2001.0 -0.020936  0.166652    0.010790  6.133715e-03  0.178737
         2002.0  0.089008  0.148092    0.009589  5.450617e-03 -0.020936
         2003.0  0.014730  0.152682    0.009886  5.619546e-03  0.089008
         2004.0  0.004440  0.163959    0.010616  6.034601e-03  0.014730
         2005.0  0.049995  0.195587    0.012664  7.198716e-03  0.004440
         2006.0  0.041857  0.181849    0.011774  6.693052e-03  0.049995
         2007.0  0.040175  0.172052    0.011140  6.332481e-03  0.041857
...                   ...       ...         ...           ...       ...
61.0     2008.0  0.083639  0.219221    0.019276  1.362005e-04  0.059592
         2009.0  0.019239  0.228180    0.020064  1.417668e-04  0.083639
         2010.0 -0.127043  0.136450    0.011998  8.477577e-05  0.019239
         2011.0  0.030572  0.007000    0.000616  4.349063e-06 -0.127043
         2012.0  0.062769 -0.016053   -0.001412 -9.973839e-06  0.030572
         2013.0  0.097100 -0.161571   -0.014207 -1.003831e-04  0.062769
         2014.0 -0.010946 -0.037843   -0.003328 -2.351173e-05  0.097100
         2015.0  0.009390 -0.002121   -0.000186 -1.317665e-06 -0.010946
         2016.0 -0.016490  0.086650    0.007619  5.383503e-05  0.009390
         2017.0 -0.014354  0.027724    0.002438  1.722499e-05 -0.016490
62.0     1998.0  0.041089  0.081466    0.016378  0.000000e+00  0.017472
         1999.0  0.033093  0.152657    0.030691  0.000000e+00  0.041089
         2000.0  0.033346  0.166705    0.033515  0.000000e+00  0.033093
         2001.0  0.017094  0.200869    0.040383  0.000000e+00  0.033346
         2002.0  0.026542  0.178499    0.035886  0.000000e+00  0.017094
         2003.0  0.036464  0.184031    0.036998  0.000000e+00  0.026542
         2004.0  0.030662  0.197623    0.039731  0.000000e+00  0.036464
         2005.0  0.053723  0.235746    0.047395  0.000000e+00  0.030662
         2006.0  0.037599  0.219186    0.044066  0.000000e+00  0.053723
         2007.0  0.071918  0.207378    0.041692  0.000000e+00  0.037599
         2008.0  0.077826  0.227411    0.045719  0.000000e+00  0.071918
         2009.0  0.014656  0.236705    0.047588  0.000000e+00  0.077826
         2010.0  0.033594  0.141548    0.028457  0.000000e+00  

In [430]:
mod = PanelOLS(data.yt, data[['yt_1','upstream','downstream','ownshock']])
res = mod.fit(cov_type='clustered', cluster_entity=True)

In [431]:
mod

PanelOLS 
Num exog: 4, Constant: False
Entity Effects: False, Time Effects: False, Num Other Effects: 0
id: 0x1a1e372a90

In [432]:
res

Dep. Variable:,yt,R-squared:,0.1686
Estimator:,PanelOLS,R-squared (Between):,0.7775
No. Observations:,1240,R-squared (Within):,0.0369
Date:,"Tue, Mar 24 2020",R-squared (Overall):,0.1686
Time:,13:48:45,Log-likelihood,1213.3
Cov. Estimator:,Clustered,,
,,F-statistic:,62.682
Entities:,62,P-value,0.0000
Avg Obs:,20.000,Distribution:,"F(4,1236)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,4.556e+06


In [433]:
# probamos con yt_2, upstream_2

In [434]:
ac=[]
for i,val in zip(range(0,len(data)),data["yt_1"]):
    year=data.index[i][1]
    if year==1998.0:
        ac.append(np.nan)
        anterior=val
    else:
        ac.append(anterior)
        anterior=val
data["yt_2"]=ac
data=data.dropna()

In [435]:
ac=[]
for i,val in zip(range(0,len(data)),data["upstream"]):
    year=data.index[i][1]
    if year==1998.0:
        ac.append(np.nan)
        anterior=val
    else:
        ac.append(anterior)
        anterior=val
data["upstream_2"]=ac
data=data.dropna()

/Users/ivanrendobarreiro/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [436]:
data # ya tenemos la yt_2 y upstream_2, upstream_3 si queremos (es la sguiente celda)

yt  upstream  downstream      ownshock      yt_1  \
industry year                                                             
1.0      1999.0 -0.024136  0.129322    0.012447  2.623307e-05  0.024989   
         2000.0  0.066981  0.141223    0.013593  2.864720e-05 -0.024136   
         2001.0  0.069624  0.170165    0.016378  3.451797e-05  0.066981   
         2002.0  0.011962  0.151214    0.014554  3.067379e-05  0.069624   
         2003.0  0.060401  0.155900    0.015005  3.162445e-05  0.011962   
         2004.0 -0.028194  0.167415    0.016114  3.396020e-05  0.060401   
         2005.0 -0.048642  0.199711    0.019222  4.051136e-05 -0.028194   
         2006.0 -0.065902  0.185682    0.017872  3.766569e-05 -0.048642   
         2007.0  0.117219  0.175679    0.016909  3.563655e-05 -0.065902   
         2008.0 -0.005149  0.192650    0.018542  3.907905e-05  0.117219   
         2009.0 -0.074567  0.200523    0.019300  4.067614e-05 -0.005149   
         2010.0  0.080842  0.119912    0.011541  2.432411e-05 -0.074567   
         2011.0 -0.044564  0.006152    0.000592  1.247846e-06  0.080842   
         2012.0 -0.013655 -0.014108   -0.001358 -2.861723e-06 -0.044564   
         2013.0  0.079022 -0.141987   -0.013666 -2.880221e-05 -0.013655   
         2014.0 -0.028297 -0.033256   -0.003201 -6.746053e-06  0.079022   
         2015.0  0.131375 -0.001864   -0.000179 -3.780683e-07 -0.028297   
         2016.0  0.063875  0.076147    0.007329  1.544651e-05  0.131375   
         2017.0  0.033884  0.024364    0.002345  4.942245e-06  0.063875   
2.0      1999.0  0.063053  0.126652    0.008201  4.661519e-03  0.167134   
         2000.0  0.178737  0.138308    0.008955  5.090500e-03  0.063053   
         2001.0 -0.020936  0.166652    0.010790  6.133715e-03  0.178737   
         2002.0  0.089008  0.148092    0.009589  5.450617e-03 -0.020936   
         2003.0  0.014730  0.152682    0.009886  5.619546e-03  0.089008   
         2004.0  0.004440  0.163959    0.010616  6.034601e-03  0.014730   
         2005.0  0.049995  0.195587    0.012664  7.198716e-03  0.004440   
         2006.0  0.041857  0.181849    0.011774  6.693052e-03  0.049995   
         2007.0  0.040175  0.172052    0.011140  6.332481e-03  0.041857   
         2008.0 -0.195385  0.188672    0.012216  6.944201e-03  0.040175   
         2009.0 -0.413250  0.196383    0.012715  7.227997e-03 -0.195385   
...                   ...       ...         ...           ...       ...   
61.0     2007.0  0.059592  0.199910    0.017578  1.242025e-04  0.071712   
         2008.0  0.083639  0.219221    0.019276  1.362005e-04  0.059592   
         2009.0  0.019239  0.228180    0.020064  1.417668e-04  0.083639   
         2010.0 -0.127043  0.136450    0.011998  8.477577e-05  0.019239   
         2011.0  0.030572  0.007000    0.000616  4.349063e-06 -0.127043   
         2012.0  0.062769 -0.016053   -0.001412 -9.973839e-06  0.030572   
         2013.0  0.097100 -0.161571   -0.014207 -1.003831e-04  0.062769   
         2014.0 -0.010946 -0.037843   -0.003328 -2.351173e-05  0.097100   
         2015.0  0.009390 -0.002121   -0.000186 -1.317665e-06 -0.010946   
         2016.0 -0.016490  0.086650    0.007619  5.383503e-05  0.009390   
         2017.0 -0.014354  0.027724    0.002438  1.722499e-05 -0.016490   
62.0     1999.0  0.033093  0.152657    0.030691  0.000000e+00  0.041089   
         2000.0  0.033346  0.166705    0.033515  0.000000e+00  0.033093   
         2001.0  0.017094  0.200869    0.040383  0.000000e+00  0.033346   
         2002.0  0.026542  0.178499    0.035886  0.000000e+00  0.017094   
         2003.0  0.036464  0.184031    0.036998  0.000000e+00  0.026542   
         2004.0  0.030662  0.197623    0.039731  0.000000e+00  0.036464   
         2005.0  0.053723  0.235746    0.047395  0.000000e+00  0.030662   
         2006.0  0.037599  0.219186    0.044066  0.000000e+00  0.053723   
         2007.0  0.071918  0.207378    0.041692  0.000000e+00  0.037599   
         2008.0  0.077826  0.227411    0.045719 

In [437]:
mod = PanelOLS(data.yt, data[['yt_1','upstream','ownshock']])
res = mod.fit(cov_type='clustered', cluster_entity=True)

In [438]:
res

Dep. Variable:,yt,R-squared:,0.1563
Estimator:,PanelOLS,R-squared (Between):,0.7564
No. Observations:,1178,R-squared (Within):,0.0367
Date:,"Tue, Mar 24 2020",R-squared (Overall):,0.1563
Time:,13:49:00,Log-likelihood,1134.7
Cov. Estimator:,Clustered,,
,,F-statistic:,72.549
Entities:,62,P-value,0.0000
Avg Obs:,19.000,Distribution:,"F(3,1175)"
Min Obs:,19.000,,
Max Obs:,19.000,F-statistic (robust):,96.818
